In [ ]:
"""
PySpark oferă:
    Un obiectul de tip Spark Context, folosit pentru interacțiunea cu Spark,.
    Un obiect de bază pentru codul SQL și citirea datelor, numit Spark Session.
    Un obiect de lucru cu date, numit Data Frame, având o interfață similară cu cel de Pandas.
    Funcții gata implementate pentru transformări și expresii.
    Abilitatea de a construi propriile funcții de transformare în Python.
"""


'\nPySpark oferă:\n    Un obiectul de tip Spark Context, folosit pentru interacțiunea cu Spark,.\n    Un obiect de bază pentru codul SQL și citirea datelor, numit Spark Session.\n    Un obiect de lucru cu date, numit Data Frame, având o interfață similară cu cel de Pandas.\n    Funcții gata implementate pentru transformări și expresii.\n    Abilitatea de a construi propriile funcții de transformare în Python.\n'

In [2]:
"""
Pregătire mediu de lucru
Stabilirea conexiunii dintre Google drive si Colab notebook
"""
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Instalarea PySpark

!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://dlcdn.apache.org/spark/
!wget -q https://dlcdn.apache.org/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz
!tar xf spark-3.4.4-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.4.4-bin-hadoop3"

import findspark
findspark.init()
findspark.find()
import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
  .builder \
  .getOrCreate()

spark

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,763 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,237 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packa

In [4]:
"""
Fiind concepute pentru Big Data, un Data Frame nu încarcă datele.
El reține doar locația și tipul lor.
La fiecare  transformare de date, nou Data Frame va fi creat care reține locația și tipul datelor, precum și lanțul de transformări.
Transformările vor fi efectuate doar în momentul scrierii sau afișării datelor de către executori.

Crearea unui Data Frame Spark dintr-o listă de Python.
"""

data = [
    ['Vali', 23, 'Programator', 4, None, ['Sport', 'Boardgames']],
    ['Vlad', 34, 'Instalator', 11, None, ['Alergare']],
    ['Bea', 29, 'Reporter', 7, True, None]
]

data_df = spark.createDataFrame(data)

data_df.show()


+----+---+-----------+---+----+-------------------+
|  _1| _2|         _3| _4|  _5|                 _6|
+----+---+-----------+---+----+-------------------+
|Vali| 23|Programator|  4|NULL|[Sport, Boardgames]|
|Vlad| 34| Instalator| 11|NULL|         [Alergare]|
| Bea| 29|   Reporter|  7|true|               NULL|
+----+---+-----------+---+----+-------------------+



In [30]:
"""
Citirea fișierelor de tip JSON Lines.
"""
path_json = '/content/drive/MyDrive/Colab Notebooks/Data/Data/practice/json'
data_df_json = spark.read.format('json').load(path_json)

#Display rezultate
data_df_json.show()


+--------------------+-------+-----------+--------------------+------+-------+
|               extra|inactiv|       nume|            ocupatie|varsta|vechime|
+--------------------+-------+-----------+--------------------+------+-------+
|            [PV, EV]|   NULL|     Andrei|Specialist marketing|    38|     13|
|   [3D Printer, WII]|   NULL|  Alexandru|      Specialist HR |    34|      8|
| [AC, EV, 5G Router]|   NULL|     Adrian|       Inginer civil|    45|     23|
|              [XBOX]|   NULL|      Alin |   Vânzător  retail |    26|      2|
|[5G Router, 3D Pr...|   NULL|      Anton|     Manager proiect|    40|     15|
|            [EV, AC]|   NULL|        Ana|  Muncitor alimentar|    35|      9|
|[PC, 3D Printer, AC]|  false|     Bogdan|           Farmacist|    50|     32|
|                NULL|   true|  Cătălin  |        Medic primar|    20|      0|
|      [AC, XBOX, PV]|   NULL|     Cosmin|           Farmacist|    49|     31|
|[XBOX, 3D Printer...|  false|   Cristian|    Asiste

In [31]:
"""
Citirea fișierelor de tip Parquet.
"""
path_parq = '/content/drive/MyDrive/Colab Notebooks/Data/Data/practice/parquet'
data_df = spark.read.format('parquet').load(path_parq)

#Display rezultate
data_df.show()


+------------+------+--------------------+-------+-------+--------------------+
|        nume|varsta|            ocupatie|vechime|inactiv|               extra|
+------------+------+--------------------+-------+-------+--------------------+
|       Daria|    33|   Inginer  mecanic |      7|   NULL|                [EV]|
|     Delia  |    20|        Medic primar|      0|   true|                NULL|
|       Diana|    45|   Asistent  social |     23|   NULL|[XBOX, 3D Printer...|
|       Doina|    37|Specialist  marke...|     11|   NULL|          [WII, PS5]|
|      Elena |    25|            Contabil|      1|   NULL|                NULL|
|     Eliza  |    23|          Instalator|      0|   NULL|                NULL|
|     Emilia |    25|            Contabil|      1|   NULL|                NULL|
|   Eugenia  |    24|        Programator |      1|   NULL|                NULL|
|     Felicia|    34|      Specialist  HR|      8|   NULL|          [WII, PS5]|
|    Florina |    28|               Șofe

In [32]:
"""
Schema Datelor

Întrucât Spark SQL a fost conceput pentru data structurate, el încearcă să determine tipurile de coloane, numită schema datelor,
 în mod automat, din sursele de date la citire. Acest proces nu este necesar pentru formate structurate ca Parquet,
  dar necesar pentru obiecte Python sau date semi-structurate, JSON și CSV.

Pentru a dezactiva determinarea automata a tipurilor de date, la citire, se poate furniza schema:
"""
from pyspark.sql import types as T

data_schema = T.StructType([
    T.StructField('nume', T.StringType(), False),
    T.StructField('varsta', T.IntegerType(), False),
    T.StructField('ocupatie', T.StringType(), False),
    T.StructField('vechime',T.IntegerType(),True),
    T.StructField('inactiv', T.BooleanType(), True),
    T.StructField('extra', T.ArrayType(T.StringType()), True)
]);

# Dacă se furnizează la citire schema, Spark va ignora sau pune NULL pe coloanele care nu se potrivesc sau nu există

data_df = spark.read.format('parquet').schema(data_schema).load(path_parq);

data_df.show()

+------------+------+--------------------+-------+-------+--------------------+
|        nume|varsta|            ocupatie|vechime|inactiv|               extra|
+------------+------+--------------------+-------+-------+--------------------+
|       Daria|    33|   Inginer  mecanic |      7|   NULL|                [EV]|
|     Delia  |    20|        Medic primar|      0|   true|                NULL|
|       Diana|    45|   Asistent  social |     23|   NULL|[XBOX, 3D Printer...|
|       Doina|    37|Specialist  marke...|     11|   NULL|          [WII, PS5]|
|      Elena |    25|            Contabil|      1|   NULL|                NULL|
|     Eliza  |    23|          Instalator|      0|   NULL|                NULL|
|     Emilia |    25|            Contabil|      1|   NULL|                NULL|
|   Eugenia  |    24|        Programator |      1|   NULL|                NULL|
|     Felicia|    34|      Specialist  HR|      8|   NULL|          [WII, PS5]|
|    Florina |    28|               Șofe

In [33]:
"""
Fie prin detectare automată, fie prin furnizarea la citire, fiecare Data Frame va avea întotdeauna o structură a datelor foarte bine definită. Pentru fiecare coloană avem numele, tipul de dată și dacă se permit valori de ”NULL”.

Dacă avem de lucrat cu date care nu au o schemă bine definită, fie date nestructurate, fie date semi-structurate cu multe abateri, atunci NU folosim Data Frame-ul din Spark SQL.

Pentru a afișa structură a datelor / schema datelor pentru un Data Frame, folosim:
"""
data_df.printSchema()



root
 |-- nume: string (nullable = true)
 |-- varsta: integer (nullable = true)
 |-- ocupatie: string (nullable = true)
 |-- vechime: integer (nullable = true)
 |-- inactiv: boolean (nullable = true)
 |-- extra: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [25]:
"""
Data Frame – Colectarea Datelor

Alte metode pe care Spark le ofertă, nu numai pentru depănarea datelor, dar și pentru rarele situații când
 am redus datele și vrem să prelucrăm cu alte librării, sunt cele de colectare a datelor în obiecte de Python pe mașina Master.

Atenție! Datele sunt transferate mai întâi de la executori pe Spark Driver la colectare.
Dacă sunt prea multe date, există pericolul ca procesul de Spark Driver să nu mai facă față și să fie terminat de către sistem.

Pentru a colecta datele într-o listă de Python, folosim:

"""

data_list = data_df.collect()
data_list

[Row(nume='Daria', varsta=33, ocupatie='Inginer  mecanic ', vechime=7, inactiv=None, extra=['EV']),
 Row(nume='  Delia  ', varsta=20, ocupatie='Medic primar', vechime=0, inactiv=True, extra=None),
 Row(nume='Diana', varsta=45, ocupatie='Asistent  social ', vechime=23, inactiv=None, extra=['XBOX', '3D Printer', 'PV']),
 Row(nume='Doina', varsta=37, ocupatie='Specialist  marketing ', vechime=11, inactiv=None, extra=['WII', 'PS5']),
 Row(nume=' Elena ', varsta=25, ocupatie='Contabil', vechime=1, inactiv=None, extra=None),
 Row(nume='  Eliza  ', varsta=23, ocupatie='Instalator', vechime=0, inactiv=None, extra=None),
 Row(nume=' Emilia ', varsta=25, ocupatie='Contabil', vechime=1, inactiv=None, extra=None),
 Row(nume='  Eugenia  ', varsta=24, ocupatie='Programator ', vechime=1, inactiv=None, extra=None),
 Row(nume='Felicia', varsta=34, ocupatie='Specialist  HR', vechime=8, inactiv=None, extra=['WII', 'PS5']),
 Row(nume=' Florina ', varsta=28, ocupatie='Șofer', vechime=3, inactiv=None, extra

In [ ]:
"""
Pentru a colecta datele într-un obiect de Data Frame, dar a librăriei Pandas, folosim:

"""

data_pandas_pdf = data_df.toPandas()
data_pandas_pdf

,nume,varsta,post,concediu,extra
0,Vali,23,None,None,"[3D Printer, XBOX]"
1,Vlad,34,None,None,[EV]
2,Bea,29,None,None,None


In [28]:
"""
Scrierea Datelor

"""

# Scrierea fișierelor de tip JSON Lines.
path_output_json = '/content/drive/MyDrive/Colab Notebooks/Data/Data/output/json'
data_df.write.format('json').mode('overwrite').save(path_output_json)



In [29]:
# Scrierea fișierelor de tip Parquet.
path_output_parq = '/content/drive/MyDrive/Colab Notebooks/Data/Data/output/parquet'
data_df.write.format('parquet').mode('overwrite').save(path_output_parq)
